In [1]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sympy import *



In [13]:
#load the iris dataset
dataset = pd.read_csv('face_feature_vectors.csv')  

X = dataset.iloc[:,2:].values
y = dataset.iloc[:,1:2].values

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

print(X.shape)

print(y.shape)


(800, 128)
(800,)


C:\Users\nagab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## split first 5 data points of each class into training set and remaining 395 points into test set

In [4]:
# seperate data into different classes
male_indices = np.where(y==1)[0]
female_indices = np.where(y==0)[0]

# split first 5 data points of each class into testing set and remaining 395 points into training set

X_test = np.concatenate((X[male_indices[:5]], X[female_indices[:5]]),axis = 0)
y_test = np.concatenate((y[male_indices[:5]], y[female_indices[:5]]),axis = 0)
X_train = np.concatenate((X[male_indices[5:]], X[female_indices[5:]]),axis = 0)
y_train = np.concatenate((y[male_indices[5:]], y[female_indices[5:]]),axis = 0)

print(X_test.shape)
print(y_test.shape)
print(X_train.shape)
print(y_train.shape)



(10, 128)
(10,)
(790, 128)
(790,)


In [6]:

mean_males = np.mean(X_train[y_train == 1], axis=0)
print("mean_vector of males:\n",mean_males)
cov_males = np.cov(X_train[y_train == 1], rowvar=False)
print("covariance_mat of males:\n",cov_males)
inv_cov_males = np.linalg.inv(cov_males)
print("inv_cov of males:\n",inv_cov_males)
#calculate the determinant of inverse covariance matrix
cov_male = Matrix(cov_males)
determinant_males = cov_male.det()
print("determinant:\n",determinant_males)


print("\n\n\n")

mean_females= np.mean(X_train[y_train == 0], axis=0)
print("mean_vector of females:\n",mean_females)
cov_females = np.cov(X_train[y_train == 0], rowvar=False)
print("covariance_mat of females:\n",cov_females)
inv_cov_females = np.linalg.inv(cov_females)
print("inv_cov of females:\n",inv_cov_females)
cov_female = Matrix(cov_females)
determinant_females = cov_female.det()
print("determinant:\n",determinant_females)

print("\n\n\n")

mean_vector of males:
 [-9.12682951e-02  9.26627972e-02  4.37648293e-02 -3.02703433e-02
 -9.34554573e-02 -1.38453392e-02 -1.03896107e-02 -1.05276523e-01
  1.29707273e-01 -5.07738911e-02  1.99340208e-01 -3.32230868e-02
 -2.39884129e-01 -5.99004539e-02 -1.96462208e-02  1.04802457e-01
 -1.49386262e-01 -1.12444197e-01 -1.22064315e-01 -9.01498322e-02
  1.27200896e-02  2.90510499e-02  3.21906934e-02  4.80791709e-03
 -1.39520250e-01 -3.08623786e-01 -7.40319441e-02 -9.73655389e-02
  6.60231589e-02 -9.69972284e-02  2.05003577e-02  4.02396851e-02
 -1.82002440e-01 -7.43015988e-02  1.88982819e-02  4.76713457e-02
 -4.52486882e-02 -5.93617062e-02  2.06963378e-01  2.47188771e-02
 -1.52393851e-01  2.50209736e-02  4.75442619e-02  2.56151338e-01
  1.93161357e-01  6.57204423e-03  2.36197253e-02 -5.99630446e-02
  1.16941670e-01 -2.43271693e-01  6.05811867e-02  1.34513861e-01
  1.43770173e-01  6.86746338e-02  8.50765823e-02 -1.28681023e-01
  1.94651526e-02  1.31400411e-01 -1.88280166e-01  8.84326692e-02
  

In [7]:
#print the dimensions of mean vector and covariance matrix
print(cov_males.shape)

(128, 128)


In [9]:
def multivariate_normal_density(x, mean_vector, inv_cov,det_cov):
    p = x.shape[0]
    exponent = -0.5 * np.dot(np.dot((x - mean_vector).T, inv_cov), (x - mean_vector))
    coef = 1 / ((2 * np.pi) ** (p / 2) * ((det_cov) ** 0.5))
    return coef * np.exp(exponent)

In [10]:
# finding multivariate_normal_density for each sample in test set for each class
male_density = np.zeros(X_test.shape[0])
female_density = np.zeros(X_test.shape[0])
for i in range(X_test.shape[0]):
    male_density[i] = multivariate_normal_density(X_test[i], mean_males, inv_cov_males,determinant_males)

    female_density[i] = multivariate_normal_density(X_test[i], mean_females, inv_cov_females,determinant_females)
    
  
    
print("male_density: ",male_density)
print("female_density: ",female_density)

# finding the predicted class for each sample in test set
y_pred = np.zeros(X_test.shape[0], dtype=np.int8)

for i in range(X_test.shape[0]):
    if male_density[i] > female_density[i]:
        y_pred[i] = 1

    else:
        y_pred[i] = 0
    
print("y-predicted matrix: ",y_pred)


male_density:  [            inf             inf             inf             inf
             inf             inf 3.98044533e+237 1.31107317e+284
 8.99090761e+242 2.09410227e+240]
female_density:  [1.14418525e+305 5.02766248e+302 7.31230319e+286             inf
 2.44385654e+281 8.49353687e+297             inf             inf
             inf             inf]
y-predicted matrix:  [1 1 1 0 1 1 0 0 0 0]


In [11]:
print("y-predicted matrix\n",y_pred)
print("y-test matrix\n",y_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
print("percent accuracy score : ",accuracy_score(y_test, y_pred)*100)

y-predicted matrix
 [1 1 1 0 1 1 0 0 0 0]
y-test matrix
 [1 1 1 1 1 0 0 0 0 0]
[[4 1]
 [1 4]]
percent accuracy score :  80.0


In [12]:
apriori_prob = 1/3
g_male = np.zeros(10)
g_female = np.zeros(10)

#calculation of discriminant function
def discriminant_function(density, apriori_prob):
    return np.log(density) + np.log(apriori_prob)
    
for i in range(10):
    
    g_male[i] = discriminant_function(male_density[i], apriori_prob)
    g_female[i] = discriminant_function(female_density[i], apriori_prob)

print("g_males: ",g_male)
print("g_females: ",g_female)


g_males:  [         inf          inf          inf          inf          inf
          inf 545.99544846 653.10640014 558.32319402 552.26093498]
g_females:  [701.32453389 695.89704096 659.43028261          inf 646.82137618
 684.90846584          inf          inf          inf          inf]
